<h1>Lab | Data cleaning and wrangling</h1><br>

For this lab, we will be using the same dataset we used in the previous labs. We recommend using the same notebook since you will be reusing the same variables you previous created and used in labs.

<h2>Instructions</h2><br>

So far we have worked on EDA. This lab will focus on data cleaning and wrangling from everything we noticed before.

* 1. We will start with removing outliers. So far, we have discussed different methods to remove outliers. Use the one you feel more comfortable with, define a function for that. Use the function to remove the outliers and apply it to the dataframe.
* 2. Create a copy of the dataframe for the data wrangling.
* 3. Normalize the continuous variables. You can use any one method you want.
* 4. Encode the categorical variables
* 5. The time variable can be useful. Try to transform its data into a useful one. Hint: Day week and month as integers might be useful.
* 6. Since the model will only accept numerical data, check and make sure that every column is numerical, if some are not, change it using encoding.

<h2>Hint for Categorical Variables</h2><br>

* You should deal with the categorical variables as shown below (for ordinal encoding, dummy code has been provided as well):<br>

In [1]:
# One hot to state
# Ordinal to coverage
# Ordinal to employmentstatus
# Ordinal to location code
# One hot to marital status
# One hot to policy type
# One hot to policy
# One hot to renew offer type
# One hot to sales channel
# One hot vehicle class
# Ordinal vehicle size

# data["coverage"] = data["coverage"].map({"Basic" : 0, "Extended" : 1, "Premium" : 2})
# given that column "coverage" in the dataframe "data" has three categories:
# "basic", "extended", and "premium" and values are to be represented in the same order.

**Copy paste from previous LAB**

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [3]:
customer_df = pd.read_csv("files_for_lab/we_fn_use_c_marketing_customer_value_analysis.csv")
customer_df.head()

,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Income,...,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2/24/11,Employed,F,56274,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,1/31/11,Unemployed,F,0,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2/19/11,Employed,F,48767,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,WW63253,California,7645.861827,No,Basic,Bachelor,1/20/11,Unemployed,M,0,...,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,HB64268,Washington,2813.692575,No,Basic,Bachelor,2/3/11,Employed,M,43836,...,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize


In [4]:
customer_df.columns = customer_df.columns.str.lower().str.replace(' ','_')

In [5]:
# Convert dates in datetime format

customer_df['effective_to_date'] = pd.to_datetime(customer_df['effective_to_date'], errors='coerce')

# Transform the column into a column that gives us only the month and the year

customer_df['effective_to_date'] = customer_df['effective_to_date'].apply(lambda x: x.strftime('%B-%Y'))

In [6]:
customer_df.drop(["customer"], axis=1, inplace = True)

**Here starts this Lab | Data cleaning and wrangling**

## 1. Removing outliers

In [7]:
# Creating a function that removes all the outliers automatically from the dataframe

def outliers_remover(df):
    print("Old Shape: ", df.shape)
    continuous_variables = df.select_dtypes(include='number') # Getting only numerical features
    for column in continuous_variables.columns:
        q75,q25 = np.percentile(df.loc[:,column],[75,25]) # Calculate the interquartiles
        intr_qr = q75-q25 # Interquartile range
        max = q75+(1.5*intr_qr) # Setting the upper edge
        min = q25-(1.5*intr_qr) # Setting the lower edge
        df.loc[df[column] < min,column] = np.nan # Transforming outliers into nans
        df.loc[df[column] > max,column] = np.nan
        # df = df.dropna().reset_index(drop=True) # Dropping the nans from the database
        
# Doing the sanity check I saw that the feature customer_lifetime_value still has nans
# I tried everything I could but it has no sense that it removes ALL the outliers but the first feature
# I sincerely don't get how it's interating throughout all the loop, calculating the outliers, transforming them into nans
# but not removing the nans. Only for the first feature, for the rest of the features it work perfectly. I'll do it manually.

    print("New Shape: ", df.shape)

In [8]:
outliers_remover(customer_df)

Old Shape:  (9134, 23)
New Shape:  (9134, 23)


In [9]:
customer_df.isnull().sum() # Checking the nans

state                               0
customer_lifetime_value           817
response                            0
coverage                            0
education                           0
effective_to_date                   0
employmentstatus                    0
gender                              0
income                              0
location_code                       0
marital_status                      0
monthly_premium_auto              430
months_since_last_claim             0
months_since_policy_inception       0
number_of_open_complaints        1882
number_of_policies                416
policy_type                         0
policy                              0
renew_offer_type                    0
sales_channel                       0
total_claim_amount                453
vehicle_class                       0
vehicle_size                        0
dtype: int64

In [10]:
# Removing the nans manually

customer_df = customer_df.dropna().reset_index(drop=True)
customer_df.isnull().sum()

state                            0
customer_lifetime_value          0
response                         0
coverage                         0
education                        0
effective_to_date                0
employmentstatus                 0
gender                           0
income                           0
location_code                    0
marital_status                   0
monthly_premium_auto             0
months_since_last_claim          0
months_since_policy_inception    0
number_of_open_complaints        0
number_of_policies               0
policy_type                      0
policy                           0
renew_offer_type                 0
sales_channel                    0
total_claim_amount               0
vehicle_class                    0
vehicle_size                     0
dtype: int64

In [11]:
customer_df.shape # Checking the new shape

(5911, 23)

## 2. Create a copy of the dataframe for the data wrangling

In [12]:
customer_df_new = customer_df.copy()
customer_df_new

,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,location_code,...,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size
0,Washington,2763.519279,No,Basic,Bachelor,February-2011,Employed,F,56274.0,Suburban,...,5.0,0.0,1.0,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,Nevada,12887.431650,No,Premium,Bachelor,February-2011,Employed,F,48767.0,Suburban,...,38.0,0.0,2.0,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
2,California,7645.861827,No,Basic,Bachelor,January-2011,Unemployed,M,0.0,Suburban,...,65.0,0.0,7.0,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
3,Washington,2813.692575,No,Basic,Bachelor,February-2011,Employed,M,43836.0,Rural,...,44.0,0.0,1.0,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize
4,Oregon,8256.297800,Yes,Basic,Bachelor,January-2011,Employed,F,62902.0,Rural,...,94.0,0.0,2.0,Personal Auto,Personal L3,Offer2,Web,159.383042,Two-Door Car,Medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5906,California,5032.165498,No,Basic,College,February-2011,Employed,M,66367.0,Suburban,...,48.0,0.0,3.0,Personal Auto,Personal L3,Offer2,Call Center,307.200000,Two-Door Car,Small
5907,California,4100.398533,No,Premium,College,January-2011,Employed,F,47761.0,Suburban,...,58.0,0.0,1.0,Personal Auto,Personal L2,Offer1,Branch,541.282007,Four-Door Car,Large
5908,California,3096.511217,Yes,Extended,College,February-2011,Employed,F,21604.0,Suburban,...,28.0,0.0,1.0,Corporate Auto,Corporate L3,Offer1,Branch,379.200000,Four-Door Car,Medsize
5909,California,7524.442436,No,Extended,College,February-2011,Employed,M,21941.0,Suburban,...,3.0,0.0,3.0,Personal Auto,Personal L2,Offer3,Branch,691.200000,Four-Door Car,Large


## 3. Normalize the continuous variables

In [13]:
# First of all, splitting between target and explanative variables

X = customer_df_new.drop(columns="total_claim_amount", axis=1)
y = customer_df_new.total_claim_amount

In [14]:
# Normalizing numerical features using MinMaxScaler

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

In [15]:
num_X = X.select_dtypes(include='number') # Getting numerical features from the explanatives variables

X[num_X.columns] = scaler.fit_transform(num_X) # Apply MinMax transformation and store the result back into X 

In [16]:
X # Sanity check

,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,location_code,...,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size
0,Washington,0.053337,No,Basic,Bachelor,February-2011,Employed,F,0.562847,Suburban,...,0.914286,0.050505,0.0,0.000000,Corporate Auto,Corporate L3,Offer1,Agent,Two-Door Car,Medsize
1,Nevada,0.755758,No,Premium,Bachelor,February-2011,Employed,F,0.487763,Suburban,...,0.514286,0.383838,0.0,0.142857,Personal Auto,Personal L3,Offer1,Agent,Two-Door Car,Medsize
2,California,0.392086,No,Basic,Bachelor,January-2011,Unemployed,M,0.000000,Suburban,...,0.514286,0.656566,0.0,0.857143,Corporate Auto,Corporate L2,Offer1,Call Center,SUV,Medsize
3,Washington,0.056818,No,Basic,Bachelor,February-2011,Employed,M,0.438443,Rural,...,0.342857,0.444444,0.0,0.000000,Personal Auto,Personal L1,Offer1,Agent,Four-Door Car,Medsize
4,Oregon,0.434439,Yes,Basic,Bachelor,January-2011,Employed,F,0.629140,Rural,...,0.400000,0.949495,0.0,0.142857,Personal Auto,Personal L3,Offer2,Web,Two-Door Car,Medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5906,California,0.210741,No,Basic,College,February-2011,Employed,M,0.663796,Suburban,...,0.171429,0.484848,0.0,0.285714,Personal Auto,Personal L3,Offer2,Call Center,Two-Door Car,Small
5907,California,0.146093,No,Premium,College,January-2011,Employed,F,0.477701,Suburban,...,0.457143,0.585859,0.0,0.000000,Personal Auto,Personal L2,Offer1,Branch,Four-Door Car,Large
5908,California,0.076441,Yes,Extended,College,February-2011,Employed,F,0.216081,Suburban,...,0.400000,0.282828,0.0,0.000000,Corporate Auto,Corporate L3,Offer1,Branch,Four-Door Car,Medsize
5909,California,0.383661,No,Extended,College,February-2011,Employed,M,0.219452,Suburban,...,0.971429,0.030303,0.0,0.285714,Personal Auto,Personal L2,Offer3,Branch,Four-Door Car,Large


## 4. Encode the categorical variables

In [17]:
# One hot to state
# Ordinal to coverage
# Ordinal to employmentstatus
# Ordinal to location code
# One hot to marital status
# One hot to policy type
# One hot to policy
# One hot to renew offer type
# One hot to sales channel
# One hot vehicle class
# Ordinal vehicle size

# data["coverage"] = data["coverage"].map({"Basic" : 0, "Extended" : 1, "Premium" : 2})
# given that column "coverage" in the dataframe "data" has three categories:
# "basic", "extended", and "premium" and values are to be represented in the same order.

In [25]:
# Taking a look on the categorical features

categorical_X = X.select_dtypes(include="object")
categorical_X.columns

Index(['state', 'response', 'coverage', 'education', 'effective_to_date',
       'employmentstatus', 'gender', 'location_code', 'marital_status',
       'policy_type', 'policy', 'renew_offer_type', 'sales_channel',
       'vehicle_class', 'vehicle_size'],
      dtype='object')

In [26]:
# To begin with, manually selecting the features that are going to be encoded with the OneHotEncoder

categorical_one_hot = X[['state', 'marital_status', 'policy_type', 'policy', 'renew_offer_type', 'sales_channel', 
                         'vehicle_class']]

In [27]:
# Dummifying the selected features

for feature in categorical_one_hot:
    one_hot = pd.get_dummies(X[feature]) # Dummifies the features  
    X = X.drop(feature,axis = 1) # Dropping the original features
    X = X.join(one_hot) # Joining the modified features

In [30]:
X.head() # Sanity check

,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,location_code,monthly_premium_auto,...,Offer3,Offer4,Agent,Branch,Call Center,Web,Four-Door Car,SUV,Sports Car,Two-Door Car
0,0.053337,No,Basic,Bachelor,February-2011,Employed,F,0.562847,Suburban,0.073394,...,0,0,1,0,0,0,0,0,0,1
1,0.755758,No,Premium,Bachelor,February-2011,Employed,F,0.487763,Suburban,0.431193,...,0,0,1,0,0,0,0,0,0,1
2,0.392086,No,Basic,Bachelor,January-2011,Unemployed,M,0.000000,Suburban,0.412844,...,0,0,0,0,1,0,0,1,0,0
3,0.056818,No,Basic,Bachelor,February-2011,Employed,M,0.438443,Rural,0.110092,...,0,0,1,0,0,0,1,0,0,0
4,0.434439,Yes,Basic,Bachelor,January-2011,Employed,F,0.629140,Rural,0.073394,...,0,0,0,0,0,1,0,0,0,1


In [31]:
# Checking the rest of the categorical features, wich they are going to be ordinal scaled

categorical_X = X.select_dtypes(include="object")
categorical_X.columns

Index(['response', 'coverage', 'education', 'effective_to_date',
       'employmentstatus', 'gender', 'location_code', 'vehicle_size'],
      dtype='object')

In [43]:
# Checking the unique values for each remaining categorical features

for feature in categorical_X:
    print('\033[1m',feature,'\033[0m')
    print("Unique values of {} are: {}".format(feature, categorical_X[feature].unique()))
    print("")

 response 
Unique values of response are: ['No' 'Yes']

 coverage 
Unique values of coverage are: ['Basic' 'Premium' 'Extended']

 education 
Unique values of education are: ['Bachelor' 'Master' 'College' 'High School or Below' 'Doctor']

 effective_to_date 
Unique values of effective_to_date are: ['February-2011' 'January-2011']

 employmentstatus 
Unique values of employmentstatus are: ['Employed' 'Unemployed' 'Medical Leave' 'Disabled' 'Retired']

 gender 
Unique values of gender are: ['F' 'M']

 location_code 
Unique values of location_code are: ['Suburban' 'Rural' 'Urban']

 vehicle_size 
Unique values of vehicle_size are: ['Medsize' 'Small' 'Large']



In [44]:
# Ordinal encoding the remaining categorical features

X["response"] = X["response"].map({"No" : 0, "Yes" : 1})
X["coverage"] = X["coverage"].map({"Basic" : 0, "Extended" : 1, "Premium" : 2})
X["education"] = X["education"].map({"High School or Below" : 0, "Bachelor" : 1, "College" : 2, "Master" : 3, "Doctor" : 4})
X["effective_to_date"] = X["effective_to_date"].map({"January-2011" : 0, "February-2011" : 1})
X["employmentstatus"] = X["employmentstatus"].map({"Employed" : 0, "Unemployed" : 1, "Medical Leave" : 2, "Disabled" : 3, "Retired" : 4})
X["gender"] = X["gender"].map({"F" : 0, "M" : 1})
X["location_code"] = X["location_code"].map({"Rural" : 0, "Suburban" : 1, "Urban" : 2})
X["vehicle_size"] = X["vehicle_size"].map({"Small" : 0, "Medsize" : 1, "Large" : 2})

In [45]:
X # Sanity check

,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,location_code,monthly_premium_auto,...,Offer3,Offer4,Agent,Branch,Call Center,Web,Four-Door Car,SUV,Sports Car,Two-Door Car
0,0.053337,0,0,1,1,0,0,0.562847,1,0.073394,...,0,0,1,0,0,0,0,0,0,1
1,0.755758,0,2,1,1,0,0,0.487763,1,0.431193,...,0,0,1,0,0,0,0,0,0,1
2,0.392086,0,0,1,0,1,1,0.000000,1,0.412844,...,0,0,0,0,1,0,0,1,0,0
3,0.056818,0,0,1,1,0,1,0.438443,0,0.110092,...,0,0,1,0,0,0,1,0,0,0
4,0.434439,1,0,1,0,0,0,0.629140,0,0.073394,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5906,0.210741,0,0,2,1,0,1,0.663796,1,0.027523,...,0,0,0,0,1,0,0,0,0,1
5907,0.146093,0,2,2,0,0,0,0.477701,1,0.394495,...,0,0,0,1,0,0,1,0,0,0
5908,0.076441,1,1,2,1,0,0,0.216081,1,0.165138,...,0,0,0,1,0,0,1,0,0,0
5909,0.383661,0,1,2,1,0,1,0.219452,1,0.321101,...,1,0,0,1,0,0,1,0,0,0


## 5. Time variable

>I already changed the time feature to a more useful feature, selecting the month and the year. It's already encoded also.

## 6. Check and make sure that every column is numerical

In [46]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5911 entries, 0 to 5910
Data columns (total 47 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   customer_lifetime_value        5911 non-null   float64
 1   response                       5911 non-null   int64  
 2   coverage                       5911 non-null   int64  
 3   education                      5911 non-null   int64  
 4   effective_to_date              5911 non-null   int64  
 5   employmentstatus               5911 non-null   int64  
 6   gender                         5911 non-null   int64  
 7   income                         5911 non-null   float64
 8   location_code                  5911 non-null   int64  
 9   monthly_premium_auto           5911 non-null   float64
 10  months_since_last_claim        5911 non-null   float64
 11  months_since_policy_inception  5911 non-null   float64
 12  number_of_open_complaints      5911 non-null   f

>Every column is numerical